In [69]:
from tensorflow import keras
from keras.optimizers import Adam
from keras import layers
from random import randint
import keras_tuner as kt

# Prepare the dataset

In [65]:
def prepare_datasets():
    train_ds, test_ds = keras.utils.image_dataset_from_directory(
        directory='../data/pngimages',
        labels='inferred', # Labels are sorted alphanumerically, so 10 comes before 2
        subset='both',
        seed=randint(0, 2**32),
        validation_split=0.2,
        image_size=(128, 128),
        color_mode='rgb',
    )

    reduced_train_ds, val_ds = keras.utils.split_dataset(train_ds, 0.25, shuffle=True) # 0.25 * 0.8 = 0.2, so size of val ds = size of test ds
    return reduced_train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = prepare_datasets()

Found 1125 files belonging to 15 classes.
Using 900 files for training.
Using 225 files for validation.


In [92]:
## Preferably put this in a different module..
def build_model(hp):
    model = keras.Sequential([
        layers.Resizing(128, 128), # This is already handled by the loader
        layers.RandomFlip('horizontal'),
        layers.Rescaling(1 / 255),
        layers.Conv2D(8, 3, input_shape=(128, 128, 3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(hp.get('fc_neurons'), activation='relu'),
        layers.Dropout(hp.get('dropout_rate')),
        layers.Dense(15, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

# Hyperparameter tuning and model training

In [93]:
hp = kt.HyperParameters()
hp.Choice(name='dropout_rate', values = [0.1,0.2])
hp.Choice(name='learning_rate', values=[0.001, 0.005])
hp.Choice(name='fc_neurons', values=[64, 128])

tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    hyperparameters=hp,
)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tuner.search(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

tuner.results_summary()

Trial 1 Complete [00h 00m 37s]
val_loss: 0.8908686637878418

Best val_loss So Far: 0.8908686637878418
Total elapsed time: 00h 00m 37s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
0.2               |0.2               |dropout_rate
0.005             |0.005             |learning_rate
128               |64                |fc_neurons

Epoch 1/100
7/8 [=========================>....] - ETA: 0s - loss: 19.8822 - accuracy: 0.0893